In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

このノートブックの役割
- 分類モデルの学習

#### --- ライブラリ ---

In [ ]:
import h5py
from IPython.display import display
import numpy as np
from os.path import join as pj
from os import getcwd as cwd
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data
from tqdm import tqdm
import visdom

# IO関連
from IO.logger import Logger
from IO.visdom import visualize
# 分類データセットのロード
from dataset.classification.loader import create_validation_split, load_validation_data
# 分類データセットのサンプリング
from dataset.classification.sampler import adopt_sampling
# 分類データセットを構築
from dataset.classification.dataset import insects_dataset
# モデル
from model.resnet.resnet import ResNet
from model.resnet.predict import test_classification
from model.resnet.loss import LabelSmoothingLoss
from model.mobilenet.mobilenet import MobileNet
from model.optimizer import AdamW, RAdam
# 評価関数
from evaluation.classification.evaluate import accuracy, confusion_matrix
# 統計関数
from evaluation.classification.statistics import compute_each_size_df, compute_all_size_df
# 可視化関数
from evaluation.classification.visualize import create_confusion_matrix, plot_df_distrib_size

#### --- 学習コンフィグ ---

In [ ]:
class args:
    # 実験名
    experiment_name: str = "sample"
    # テストデータの割合
    test_ratio: float = 0.2
    # パス
    all_data_path: str = pj(cwd(), "data/all_classification_data/classify_insect_std_20200806")
    model_root: str = pj(cwd(), "output_model/classification/resnet50/b20_lr1e-5", experiment_name)
    figure_root: str = pj(cwd(), "figure/classification/resnet50/b20_lr1e-5", experiment_name)
    # 学習時の設定
    model_name: str = "resnet50" # ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]の中から一つ
    bs: int = 20
    lr: float = 1e-5
    lamda: float = 0
    nepoch: int = 100
    pretrain: bool = True
    param_freeze: bool = False
    sampling: str = "OverSample" # [None, "RandomSample", "OverSample"]の中から一つ
    method_aug: str = ["All"] # dataset.classification.dataset.create_aug_seqにあるものから選択(複数可)
    optimizer: str = "AdamW" # ["Adam, AdamW", "RAdam"]の中から一つ
    activation_function: str = "ReLU" # ["ReLU", "LeakyReLU", "RReLU"]の中から一つ
    decoder: str = "Concatenate" # [None, "Concatenate", "FPN"]の中から一つ
    label_smoothing: str = "manual" # [None, "manual", "knowledge_distillation"]の中から一つ
    size_normalization: str = "uniform" # [None, "mu", "sigma", "mu_sigma", "uniform"]の中から一つ
    use_dropout: bool = True
    # テスト時の設定
    save_fig: bool = True
    save_df: bool = True
    # Visdom
    visdom: bool = False
    port: int = 8097 # defaultは8097

In [ ]:
dataset_name = args.all_data_path.split('/')[-1]
if dataset_name == 'classify_insect_std':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_resizeFAR':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_resize':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_plus_other':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera', 'Other']
elif dataset_name == 'classify_insect_std_20200806':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_20200806_DBSCAN':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_20200806':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']

#### --- 学習関連 ---

In [ ]:
def read_knowledge(knowledge_path):
    """
        knowledge(学習済みモデルの出力)を読み込む
        引数:
            - knowledge_path: str, knowledgeファイルのパス
            knowledgeは全データに対して予測値を出し, 予測値をクラスごとに分けて平均を取ったもの
    """
    knowledge = []
    with open(knowledge_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.split("\n")[0]
            line = line.split(" ")
            line = [float(element) for element in line]
            knowledge.append(line)
    
    knowledge = np.array(knowledge)
    return knowledge

In [ ]:
def define_loss(label_smoothing=None):
    """
        誤差関数を定義
        引数:
            - label_smoothing: str, ラベル平滑化の方法
            [None, "manual", "knowledge_distillation"]の中から一つ
            manualは存在確率としてターゲットクラスに0.9, その他のクラスに0.1を等分割したものを与える
            knowledge_distillationは別の学習済みモデルの出力を, 平滑化ラベルとして与える
    """
    if label_smoothing == "manual":
        print("cross_entropy_loss == LabelSmoothingLoss (manual)")
        ce = LabelSmoothingLoss(0.1, len(args.labels))
    elif label_smoothing == "knowledge_distillation":
        print("cross_entropy_loss == LabelSmoothingLoss (knowledge_distillation)")
        knowledge = read_knowledge(pj(cwd(), "data/insect_knowledge/resnet50_b20_r45_lr1e-5_crossvalid_20200806_All/knowledge.txt"))
        ce = LabelSmoothingLoss(0.1, len(args.labels), knowledge=knowledge)
    else:
        print("cross_entropy_loss == nn.CrossEntropyLoss")
        ce = torch.nn.CrossEntropyLoss().cuda()
    l2_loss = nn.MSELoss(reduction='mean').cuda()
    return ce, l2_loss

In [ ]:
def define_optimizer(lr, optimizer="AdamW"):
    """
        最適化器を定義
        引数:
            - lr: float, 学習率
            - optimizer: str, 最適化器の種類
            ["Adam, AdamW", "RAdam"]の中から一つ
    """
    if optimizer == "Adam":
        print("optimizer == Adam")
        opt = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer == "AdamW":
        print("optimizer == AdamW")
        opt = AdamW(model.parameters(), lr=lr)
    elif optimizer == "RAdam":
        print("optimizer == RAdam")
        opt = RAdam(model.parameters(), lr=lr)
    else:
        print("error! optimizer is not defined")
    return opt

In [ ]:
def train_per_epoch(train_dataloader, opt, model, ce, l2_loss, lamda=0.):
    """
        1epochの学習コード
        引数:
            - train_dataloader: データローダ
            - opt: 最適化器
            - model: モデル
            - ce: クロスエントロピー誤差
            - l2_loss: L2誤差
            - lamda: float, モデル重み正規化での重み
    """
    # set model train mode
    model.train()
    
    for x, y in train_dataloader:
        x = x.cuda()
        y = y.cuda()
        opt.zero_grad()
        out = model(x)
        if len(out.shape) == 1:
            # bsが1だとエラーとなるので, 空の次元を挿入
            out = out[None, :]

        cls_loss = ce(out, y)
        if lamda != 0:
            # モデル重みのL2正規化
            norm_loss = 0
            for param in model.parameters():
                param_target = torch.zeros(param.size()).cuda()
                norm_loss += l2_loss(param, param_target)

            norm_loss = norm_loss * lamda
            cls_loss_item = cls_loss.item()
            norm_loss_item = norm_loss.item()
            loss = cls_loss + norm_loss
        else:
            norm_loss = 0
            cls_loss_item = cls_loss.item()
            norm_loss_item = 0
            loss = cls_loss

        loss.backward()
        opt.step()
        loss_item = loss.item()
    
    return cls_loss_item, norm_loss_item, loss_item

In [ ]:
def train(train_dataloader, valid_dataloader, test_dataloader, model, lr=1e-5, lamda=0, \
          nepoch=100, visdom=False, label_smoothing=None, optimizer="Adam"):
    """
        学習コード
        引数:
            - train_dataloader: 学習データローダ
            - valid_dataloader: 学習評価データローダ
            - test_dataloader: テストデータローダ
            - model: モデル
            - lr: float, 学習率
            - lamda: float, モデル重み正規化での重み
            - nepoch: int, 何エポック学習するか
            - visdom: bool, visdomで可視化するかどうか
            - label_smoothing: str, ラベル平滑化の方法
            - optimizer: str, 最適化器の種類
    """
    # define loss
    ce, l2_loss = define_loss(label_smoothing=label_smoothing)
    
    # define optimizer
    opt = define_optimizer(lr, optimizer=optimizer)
    
    # set best acc
    best_te_acc = 0
    
    # training
    for epoch in tqdm(range(nepoch),leave=False):
        sum_cls_loss = 0
        sum_norm_loss = 0
        total_loss = 0
        
        # 1epochの学習
        epoch_cls_loss, epoch_norm_loss, epoch_loss = train_per_epoch(train_dataloader, opt, model, ce, l2_loss, lamda=lamda)
        sum_cls_loss += epoch_cls_loss
        sum_norm_loss += epoch_norm_loss
        total_loss += epoch_loss
        
        # 評価 (evalモードとtrainモードの切り替えを忘れずに)
        model.eval()
        valid_acc = accuracy(valid_dataloader, model)
        te_acc = accuracy(test_dataloader, model)
        model.train()
        
        # 最良性能ならモデルを保存
        if te_acc > best_te_acc:
            best_te_acc = te_acc
            torch.save(model.state_dict(), pj(args.model_root, "valid_" + str(valid_count + 1) + "_best.pth"))
            with open(pj(args.model_root, "valid_" + str(valid_count + 1) + "_best_accuracy.txt"), mode="w") as f:
                f.write("epoch = {}, test_acc = {}".format(epoch, te_acc))
        
        # visdomで評価指標を可視化
        if visdom:
            visualize(vis, epoch+1, sum_cls_loss, win_cls_loss)
            visualize(vis, epoch+1, sum_norm_loss, win_norm_loss)
            visualize(vis, epoch+1, total_loss, win_train_loss)
            visualize(vis, epoch+1, te_acc, win_test_acc)
            visualize(vis, epoch+1, valid_acc, win_train_acc)
        print("epoch=%s: sum_cls_loss=%f, sum_norm_loss=%f, total_loss=%f, train_acc=%f, te_acc=%f" % (epoch, sum_cls_loss, sum_norm_loss, total_loss, valid_acc, te_acc))

#### --- Visdom ---

In [ ]:
if args.visdom:
    # Visdomを起動
    vis = visdom.Visdom(port=args.port)
    
    # 窓を作成
    win_cls_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='cls_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_norm_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='norm_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_train_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_train_acc = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_accuracy',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_test_acc = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='test_accuracy',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )

#### --- コンフィグの保存 ---

In [ ]:
args_logger = Logger(args)
args_logger.save()

#### ---- メイン処理 ---

In [ ]:
if os.path.exists(args.model_root) is False:
    os.makedirs(args.model_root)
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
def train_per_valid():
    pass

In [ ]:
def create_validation_data(X, Y, train_idx, test_idx, sampling=None):
    """
        交差検証用データ作成
        引数:
            - X: np.array, 昆虫画像全体
            - Y: np.array, ラベル全体
            - train_idx: np.array, オーバーサンプリング適用前の学習インデックス
            - test_idx: np.array, テストインデックス
    """
    valid_train_idx = adopt_sampling(Y, train_idx, args.sampling)
    valid_test_idx = test_idx
    xtr, ytr, xte, yte = load_validation_data(X, Y, valid_train_idx, valid_test_idx)
    return xtr, ytr, xte, yte

In [ ]:
def create_dataloader(X, Y, xtr, ytr, xte, yte, bs=20, method_aug=None, size_normalization=None):
    """
        データローダ作成
        引数:
            - X: np.array, 昆虫画像全体
            - Y: np.array, ラベル全体
            - xtr: np.array, 学習用昆虫画像
            - ytr: np.array, 学習用ラベル
            - xte: np.array, テスト用昆虫画像
            - yte: np.array, テスト用ラベル
            - bs: int, バッチサイズ
            - method_aug: [str, ...], 学習時に適用するデータ拡張のリスト
            dataset.classification.dataset.create_aug_seqにあるものから選択(複数可)
            - size_normalization: str, 昆虫のサイズ分布の正規化方法
            [None, "mu", "sigma", "mu_sigma", "uniform"]の中から一つ 
    """
    train_dataset = insects_dataset(xtr, ytr, training=True, method_aug=method_aug, size_normalization=size_normalization)
    train_dataloader = data.DataLoader(train_dataset, bs, num_workers=bs, shuffle=True)
    valid_dataset = insects_dataset(xtr, ytr, training=False)
    valid_dataloader = data.DataLoader(valid_dataset, 1, num_workers=1, shuffle=False)
    test_dataset = insects_dataset(xte, yte, training=False)
    test_dataloader = data.DataLoader(test_dataset, 1, num_workers=1, shuffle=False)
    all_dataset = insects_dataset(X, Y, training=False)
    all_dataloader = data.DataLoader(all_dataset, 1, num_workers=1, shuffle=False)
    return train_dataloader, valid_dataloader, test_dataloader, all_dataloader

In [ ]:
valid_num = int(1.0/args.test_ratio)
with h5py.File(args.all_data_path) as f:
    X = f["X"][:]
    Y = f["Y"][:]
_, ntests = np.unique(Y, return_counts=True)
train_idxs, test_idxs = create_validation_split(Y, args.test_ratio)

valid_result = []
fail_count = np.zeros(Y.shape[0], dtype="int")
for valid_count in range(valid_num):
    # create validation data
    train_idx = train_idxs[valid_count]
    test_idx = test_idxs[valid_count]
    xtr, ytr, xte, yte = create_validation_data(X, Y, train_idx, test_idx, sampling=args.sampling)
    # create dataloader
    train_dataloader, valid_dataloader, test_dataloader, all_dataloader = \
    create_dataloader(X, Y, xtr, ytr, xte, yte, bs=args.bs, method_aug=args.method_aug, size_normalization=args.size_normalization)
    
    # create model (他のモデルもここで宣言すれば使える)
    model = ResNet(args.model_name, len(args.labels), pretrain=args.pretrain, param_freeze=args.param_freeze, \
                   use_dropout=args.use_dropout, activation_function=args.activation_function, decoder=args.decoder).cuda()
    
    # training
    train(train_dataloader, valid_dataloader, test_dataloader, model, lr=args.lr, lamda=args.lamda, \
          nepoch=args.nepoch, visdom=args.visdom, label_smoothing=args.label_smoothing, optimizer=args.optimizer)
    
    # モデルの保存
    model.load_state_dict(torch.load(pj(args.model_root, "valid_" + str(valid_count + 1) + "_best.pth")))
    
    # 交差検証ごとの評価
    model.eval()
    matrix = confusion_matrix(test_dataloader, model, args.labels) # テストデータセットの混同行列
    valid_result.extend(test_classification(test_dataloader, model)) # テストデータセットの予測の集計
    _, correct = accuracy(all_dataloader, model, return_correct=True) # 全データでの実験結果
    model.train()
    fail_count += ~correct # 予測失敗例を集計
    
    # 交差検証ごとの混同行列の集計
    df = pd.DataFrame(matrix)
    display(df)
    if valid_count == 0:
        validation_matrix = matrix
        x_all = xte
        y_all = yte
    else:
        validation_matrix += matrix
        x_all = np.concatenate([x_all, xte])
        y_all = np.concatenate([y_all, yte])

In [ ]:
df = pd.DataFrame(validation_matrix)
if args.save_df is True:
    df.to_csv(pj(args.figure_root, "validation_matrix.csv"))
df

In [ ]:
create_confusion_matrix(validation_matrix, ntests, args.labels, args.figure_root, save=args.save_fig)

In [ ]:
each_df = compute_each_size_df(valid_result, x_all, y_all)
if args.save_df is True:
    each_df.to_csv(pj(args.figure_root, "each_size_df.csv"))
each_df

In [ ]:
all_df = compute_all_size_df(each_df)
if args.save_df is True:
    all_df.to_csv(pj(args.figure_root, "all_size_df.csv"))
all_df

In [ ]:
fail_df = pd.DataFrame({"fail_count": fail_count})
if args.save_df is True:
    fail_df.to_csv(pj(args.figure_root, "fail_count.csv"))
fail_df

In [ ]:
plot_df_distrib_size(all_df, args.figure_root, save=args.save_fig)